In [ ]:

!pip install datasets transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install bitsandbytes


In [ ]:

from huggingface_hub import login
login(token="your_token_here")


In [ ]:
from datasets import load_dataset
from transformers import LlamaForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:

dataset = load_dataset('VishaalY/synthetic-code-generations', split='train')

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

dataset.jsonl:   0%|          | 0.00/459M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119217 [00:00<?, ? examples/s]

In [ ]:

eval_dataset = dataset.select(range(1000))
train_dataset = dataset.select(range(1000, len(dataset)))

In [ ]:

model_name = "meta-llama/Llama-3.2-1B"
model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="float32",
    use_cache=False

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:

model.gradient_checkpointing_enable()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else "[PAD]"
    if tokenizer.pad_token == "[PAD]":
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [ ]:

def tokenize_function(examples):
    encoding = tokenizer(examples['response'], padding="max_length", truncation=True, max_length=400)
    encoding['labels'] = encoding['input_ids']
    return encoding

In [ ]:

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/118217 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


In [ ]:
train_size = len(tokenized_train_dataset)
batch_size = 2
num_train_steps = train_size // batch_size


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    max_steps=60,
    report_to=["none"],
    gradient_accumulation_steps=4,
    fp16=False,
    dataloader_num_workers=2,
    group_by_length=True,
    warmup_ratio = 0.03,
    warmup_steps = 5,
    #save_steps = 0,
    lr_scheduler_type="linear",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)


In [ ]:
# Start training
trainer.train()


Step,Training Loss
10,2.603200
20,0.937200
30,0.845700
40,0.859700
50,0.839000
60,0.873600


TrainOutput(global_step=60, training_loss=1.1597369035085043, metrics={'train_runtime': 471.9174, 'train_samples_per_second': 1.017, 'train_steps_per_second': 0.127, 'total_flos': 1122045788160000.0, 'train_loss': 1.1597369035085043, 'epoch': 0.004060295386489367})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/without_unsloth")
tokenizer.save_pretrained("/content/drive/MyDrive/without_unsloth")


('/content/drive/MyDrive/without_unsloth/tokenizer_config.json',
 '/content/drive/MyDrive/without_unsloth/special_tokens_map.json',
 '/content/drive/MyDrive/without_unsloth/tokenizer.json')